In [29]:
import re

# Specify the input and output file names
input_file = 'ebook_numbers.txt'  # Change this to your input file name
output_file = 'five_digit_numbers.txt'

# Read the content of the input file
with open(input_file, 'r') as file:
    content = file.read()

# Use regular expression to find all 5-digit numbers
five_digit_numbers = re.findall(r'\b\d{5}\b', content)

# Write the found numbers to the output file
with open(output_file, 'w') as file:
    for number in five_digit_numbers:
        file.write(number + '\n')

print(f"Found {len(five_digit_numbers)} five-digit numbers and saved to {output_file}.")

Found 64365 five-digit numbers and saved to five_digit_numbers.txt.


In [37]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time

# Setup Chrome options
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run in headless mode (no GUI)
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# Set up the WebDriver using ChromeDriverManager
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# Step 1: Read five-digit numbers from a text file
with open('five_digit_numbers.txt', 'r') as file:
    numbers = [line.strip() for line in file if line.strip().isdigit() and len(line.strip()) == 5]

# Limit to the first 10 five-digit numbers
numbers = numbers[20:40]

# Base URL
base_url = "https://www.gutenberg.org/ebooks/"

# Initialize a list to store scraped text
scraped_texts = []

# Step 2: Loop through each number, construct the URL, and scrape the text
for number in numbers:
    url = f"{base_url}{number}"
    print(f"Opening URL: {url}")
    
    try:
        driver.get(url)
        time.sleep(2)  # Allow time for the page to load
        
        # Step 3: Click on the "Plain Text UTF-8" link
        try:
            plain_text_link = driver.find_element(By.LINK_TEXT, 'Plain Text UTF-8')
            plain_text_link.click()
        except:
            print("Not found")
        
        time.sleep(2)  # Allow time for the page to load
        
        # Get the page source after clicking the link
        text_content = driver.page_source
        
        # Append to the list of scraped texts
        scraped_texts.append(text_content)
        
    except Exception as e:
        print(f"Error occurred while processing {url}: {e}")

# Step 4: Store the scraped text in a txt file
with open('scraped_texts.txt', 'w', encoding='utf-8') as output_file:
    for text in scraped_texts:
        output_file.write(text + "\n\n")  # Add a separator between different texts

# Close the driver
driver.quit()

print("Scraping complete. Texts have been stored in 'scraped_texts.txt'.")

Opening URL: https://www.gutenberg.org/ebooks/47836
Opening URL: https://www.gutenberg.org/ebooks/41750
Opening URL: https://www.gutenberg.org/ebooks/44556
Opening URL: https://www.gutenberg.org/ebooks/38460
Opening URL: https://www.gutenberg.org/ebooks/41749
Opening URL: https://www.gutenberg.org/ebooks/34807
Opening URL: https://www.gutenberg.org/ebooks/38459
Opening URL: https://www.gutenberg.org/ebooks/30822
Opening URL: https://www.gutenberg.org/ebooks/34806
Opening URL: https://www.gutenberg.org/ebooks/27681
Opening URL: https://www.gutenberg.org/ebooks/30821
Opening URL: https://www.gutenberg.org/ebooks/24098
Opening URL: https://www.gutenberg.org/ebooks/27680
Opening URL: https://www.gutenberg.org/ebooks/20240
Opening URL: https://www.gutenberg.org/ebooks/24097
Opening URL: https://www.gutenberg.org/ebooks/17438
Opening URL: https://www.gutenberg.org/ebooks/20239
Opening URL: https://www.gutenberg.org/ebooks/14548
Opening URL: https://www.gutenberg.org/ebooks/17437
Opening URL: